# TAG 5 – Clean Python Code I

---

## 🎯 Ziel & Prinzip

### Was ist das?
Clean Code für Data Science bedeutet, Analyse-Code so zu schreiben,  
dass er strukturiert, nachvollziehbar und langfristig wartbar ist.

---

### Warum existiert es?
Damit Analysen:
- nachvollziehbar bleiben
- reproduzierbar sind
- von anderen verstanden werden
- im Team weiterentwickelt werden können

Data Science ist kein Ein-Personen-Projekt.
Sauberer Code reduziert Risiko.

---

### Wann brauche ich es?
Sobald:
- dein Code länger als 20–30 Zeilen wird
- mehrere Schritte hintereinander ausgeführt werden
- du Business-Regeln implementierst
- du dein Notebook später wiederverwenden willst

---

### Welches Problem löst es?
Clean Code verhindert:
- Skript-Chaos
- versteckte Business-Regeln
- unklare Variablen
- Magic Numbers
- schwer auffindbare Fehler
- Debug-Hölle

---

## 📚 Heute üben wir

- Funktionen statt Skript-Chaos
- sprechende Variablen
- keine Magic Numbers (Konstanten oder Parameter)

Ziel:
Ein fremder Student kann deinen Code lesen und verstehen,
ohne dass du ihn erklären musst.


## 1) Setup

In [64]:
import pandas as pd
import numpy as np


## 2) Konfiguration statt Magic Numbers

**Intuition:** Business-Regeln gehören an eine sichtbare Stelle.  
**Technisch:** Konstanten sind „Single Source of Truth“.  
**Business:** Wenn sich die Schwelle ändert (z. B. HighValue = 250 statt 200), willst du nur eine Stelle anfassen.


In [65]:
HIGH_VALUE_THRESHOLD = 200
CURRENCY = "€"


## 3) Datenquelle: reproduzierbares Mini-Dataset

### Was ist das?
Ein kleines, festes Dataset für Übungen.

### Warum?
Damit Ergebnisse reproduzierbar sind und nicht zufällig variieren.

### Wann?
Beim Lernen, bei Demos, bei Unit-Tests, bei Interview-Übungen.

Wichtig:
Das Dataset ist absichtlich klein.
Wir üben Struktur – nicht Big Data.


In [66]:
def make_dataset() -> pd.DataFrame:
    """Create a small synthetic ecommerce transaction dataset."""
    
    data = {
        "customer_id": [1, 2, 1, 3, 2, 4, 5, 3],
        "country":     ["AT", "DE", "AT", "DE", "DE", "AT", "AT", "DE"],
        "revenue":     [200.0, 50.0, 150.0, 300.0, 80.0, 400.0, 60.0, 120.0],
        "segment":     ["Premium", "Standard", "Premium", "Premium", "Standard", "Premium", "Standard", "Premium"],
        "order_date":  pd.to_datetime([
            "2025-01-05", "2025-01-06", "2025-01-09", "2025-01-10",
            "2025-01-11", "2025-01-12", "2025-01-14", "2025-01-15"
        ])
    }
    
    return pd.DataFrame(data)


df = make_dataset()

df


,customer_id,country,revenue,segment,order_date
0,1,AT,200.0,Premium,2025-01-05
1,2,DE,50.0,Standard,2025-01-06
2,1,AT,150.0,Premium,2025-01-09
3,3,DE,300.0,Premium,2025-01-10
4,2,DE,80.0,Standard,2025-01-11
5,4,AT,400.0,Premium,2025-01-12
6,5,AT,60.0,Standard,2025-01-14
7,3,DE,120.0,Premium,2025-01-15


In [67]:
df.info()


<class 'pandas.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   customer_id  8 non-null      int64         
 1   country      8 non-null      str           
 2   revenue      8 non-null      float64       
 3   segment      8 non-null      str           
 4   order_date   8 non-null      datetime64[us]
dtypes: datetime64[us](1), float64(1), int64(1), str(2)
memory usage: 527.0 bytes


---------------------------------------------------------
## 4) Datenqualität: Typen & Missing Values

### Intuition
Bevor wir KPIs berechnen, stellen wir sicher,
dass Zahlen wirklich Zahlen sind.

### Warum existiert das?
In echten Daten kommen:
- Strings wie "200€"
- leere Felder
- Sonderzeichen
- falsche Formate

vor.

### Wann brauche ich es?
Sobald Daten aus CSV, Excel oder APIs kommen.

### Problem
"200" als String kann Berechnungen verfälschen
oder stille Fehler erzeugen.


## 4.1) Clean-Funktion implementieren

In [68]:
def clean_revenue(transactions: pd.DataFrame) -> pd.DataFrame: 
    #transactions: pd.DataFrame -> pd.DataFrame bedeuett, dass die Funktion clean_revenue 
    #einen DataFrame als Eingabe erwartet und einen DataFrame als Ausgabe zurückgibt.
    """
    Ensure revenue column is numeric.
    Non-convertible values become NaN.
    """
    df = transactions.copy() 
    # Die Funktion .copy() erstellt eine Kopie des DataFrames transactions, 
    # um sicherzustellen, dass die Originaldaten nicht verändert werden.
    df["revenue"] = pd.to_numeric(df["revenue"], errors="coerce")
    # Die Funktion pd.to_numeric() versucht, die Werte in der Spalte "revenue" in numerische Werte umzuwandeln.
    # Wenn es Werte gibt, die nicht konvertierbar sind (z.B. Text), werden diese mit NaN (Not a Number) ersetzt, 
    # da errors="coerce" angegeben ist.
    return df


In [69]:
df_dirty = df.copy()
df_dirty["revenue"] = df_dirty["revenue"].astype("object")  # erlaubt Strings

df_dirty.loc[2, "revenue"] = "200€"
df_dirty.loc[4, "revenue"] = "abc"

df_dirty


,customer_id,country,revenue,segment,order_date
0,1,AT,200.0,Premium,2025-01-05
1,2,DE,50.0,Standard,2025-01-06
2,1,AT,200€,Premium,2025-01-09
3,3,DE,300.0,Premium,2025-01-10
4,2,DE,abc,Standard,2025-01-11
5,4,AT,400.0,Premium,2025-01-12
6,5,AT,60.0,Standard,2025-01-14
7,3,DE,120.0,Premium,2025-01-15


In [70]:
clean_df = clean_revenue(df_dirty)
clean_df
clean_df["revenue"]



0    200.0
1     50.0
2      NaN
3    300.0
4      NaN
5    400.0
6     60.0
7    120.0
Name: revenue, dtype: float64

--------------------------------

## 5) Feature Engineering: HighValue-Flag

### Was ist das?
Eine abgeleitete Spalte (Feature) aus einer Regel.

### Warum?
Du machst Entscheidungslogik explizit:
„Ab wann ist eine Transaktion wertvoll?“

### Wann?
- Segmentierung
- Alerts (Fraud/Risk)
- KPI-Slices
- ML-Features

### Problem gelöst
Die Regel ist sichtbar, konsistent und wiederverwendbar,
weil sie nicht „irgendwo im Code“ versteckt ist.


In [71]:
def add_high_value_flag(transactions: pd.DataFrame, threshold: float = HIGH_VALUE_THRESHOLD) -> pd.DataFrame:
    """Add a boolean HighValue flag based on revenue threshold."""
    df = transactions.copy()
    df["HighValue"] = df["revenue"] > threshold
    return df


In [72]:
clean_df = add_high_value_flag(clean_df, threshold=HIGH_VALUE_THRESHOLD)
clean_df[["customer_id", "revenue", "HighValue"]]


,customer_id,revenue,HighValue
0,1,200.0,False
1,2,50.0,False
2,1,NaN,False
3,3,300.0,True
4,2,NaN,False
5,4,400.0,True
6,5,60.0,False
7,3,120.0,False


-------------------
## 6) Filter: Nur HighValue-Zeilen anzeigen

### Intuition
Nicht mit `if` „Zeilenweise entscheiden“, sondern Tabellen filtern.

### Technisch
Boolean Indexing:
Ein True/False-Vektor entscheidet, welche Zeilen bleiben.

### Business
Du extrahierst eine Teilmenge für:
- Review
- Alerts
- Reporting


In [73]:
def get_high_value_rows(transactions: pd.DataFrame) -> pd.DataFrame:
    """Return only rows where HighValue is True."""
    return transactions[transactions["HighValue"]] 
    # Filtert den DataFrame transactions, um nur die Zeilen zurückzugeben, 
    # bei denen die Spalte "HighValue" den Wert True hat.


In [74]:
high_value_rows = get_high_value_rows(clean_df)
high_value_rows[["customer_id", "country", "revenue", "segment", "HighValue"]]
# In diesem Beispiel werden nur die Zeilen zurückgegeben, bei denen die Spalte "HighValue" den Wert True hat.

,customer_id,country,revenue,segment,HighValue
3,3,DE,300.0,Premium,True
5,4,AT,400.0,Premium,True


In [75]:
clean_df["HighValue"].value_counts(dropna=False)


HighValue
False    6
True     2
Name: count, dtype: int64

## 7) KPI: Durchschnittlicher Umsatz pro Segment

### Was ist das?
Aggregation von Transaktionsdaten zu einer KPI pro Segment.

### Warum?
Rohdaten sind nicht entscheidungsfähig.
KPIs machen Daten vergleichbar und steuerbar.

### Wann?
- Reporting
- Segmentvergleiche
- Produktentscheidungen
- Priorisierung von Maßnahmen

### Problem gelöst
Du siehst sofort, welches Segment im Schnitt wertvoller ist.

------------------------------------------------------------

# 7.1) KPI (Key Performance Indicators) berechnen

In [76]:
def mean_revenue_by_segment(transactions: pd.DataFrame) -> pd.Series:
    """Compute mean revenue per segment (NaN ignored by default)."""
    return transactions.groupby("segment")["revenue"].mean()
    
# Die Funktion mean_revenue_by_segment berechnet den durchschnittlichen Umsatz (revenue) 
# für jede Kundensegment (segment) im DataFrame transactions.
# Sie gruppiert die Daten nach der Spalte "segment" und berechnet dann den Mittelwert 
# der "revenue" für jede Gruppe. NaN-Werte werden standardmäßig ignoriert.

In [77]:
mean_rev = mean_revenue_by_segment(clean_df)
mean_rev


segment
Premium     255.0
Standard     55.0
Name: revenue, dtype: float64

-----------------------------------------------------

## 8) KPI-Ausgabe: Business-tauglich formatieren

### Intuition
Niemand will `dtype: float64` oder 6 Nachkommastellen im Report sehen.

### Technisch
Wir formatieren bewusst mit Python-Output.

### Business
Lesbarkeit = schnellere Entscheidungen und weniger Missverständnisse.


In [78]:
def print_mean_revenue_by_segment(mean_rev: pd.Series, currency: str = CURRENCY) -> None:
    print("Durchschnittlicher Umsatz pro Segment:\n")
    for segment, value in mean_rev.items():
        if pd.isna(value):
            print(f"{segment}: n/a")
        else:
            print(f"{segment}: {value:.2f} {currency}")
# Diese Funktion print_mean_revenue_by_segment nimmt eine Pandas Series mean_rev, 
# die den durchschnittlichen Umsatz pro Segment enthält,
# und ein optionales Argument currency, das die Währung angibt (Standard ist "€").
# Sie gibt den durchschnittlichen Umsatz pro Segment in einem lesbaren Format aus.



In [79]:
print_mean_revenue_by_segment(mean_rev)


Durchschnittlicher Umsatz pro Segment:

Premium: 255.00 €
Standard: 55.00 €


## 9) Mini-Zusammenfassung

- **Konstanten statt Magic Numbers** → Business-Regeln sind zentral sichtbar.
- **Funktionen trennen Verantwortlichkeiten** → Pipeline ist lesbar und wartbar.
- **Boolean Indexing statt `if`** → Tabellen-Logik korrekt umgesetzt.
- **KPI-Ausgabe formatieren** → Business-taugliche Kommunikation.


## 10) Kontrollfragen

1) Warum sind sprechende Namen wichtiger als Kommentare?  
2) Wann sind Magic Numbers gefährlich?  
3) Warum ist `if` hier schlechter als Boolean Indexing?


## 11) Mini-Aufgabe

Ändere den Schwellenwert auf **250** und beantworte:

1) Wie viele HighValue-Transaktionen bleiben übrig?  
2) Welche Kunden sind betroffen?  
3) Was ändert sich am Segment-KPI (falls überhaupt)?

**Tipp:** Du darfst nur eine Stelle ändern: `HIGH_VALUE_THRESHOLD`.


In [84]:
# 1) Threshold ändern: nur hier
HIGH_VALUE_THRESHOLD = 250

# 2) Pipeline erneut laufen lassen (keine anderen Änderungen)
df = make_dataset()
clean_df = clean_revenue(df)
clean_df = add_high_value_flag(clean_df, threshold=HIGH_VALUE_THRESHOLD)

high_value_rows = get_high_value_rows(clean_df)
mean_rev = mean_revenue_by_segment(clean_df)

print("HighValue count (True):", int(clean_df["HighValue"].sum()))
print(high_value_rows[["customer_id", "revenue", "HighValue"]])

print()
print_mean_revenue_by_segment(mean_rev)


HighValue count (True): 2
   customer_id  revenue  HighValue
3            3    300.0       True
5            4    400.0       True

Durchschnittlicher Umsatz pro Segment:

Premium: 234.00 €
Standard: 63.33 €
